### BIOSTAT 257: HW3

#### Q1: Formula

In [14]:
# load libraries
using BenchmarkTools, DelimitedFiles, Images, LinearAlgebra, Random

Consider a linear mixed effects model,
        $$\mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n $$
 where,
 - $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,
 - $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effect predictor matrix of $i$-th individual,
 - $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effect predictor matrix of $i$-th individuaL,
 - $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$, 
 - $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and
 - $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

#### Q1: Formula

Write down the log-likelihood of the  𝑖 -th datum  (𝐲𝑖,𝐗𝑖,𝐙𝑖)  given parameters  (𝜷,𝚺,𝜎2).

Start by finding the distribution of $Y_i$: 

- $E(Y_i) = X_i\beta$ 
- $Var(Y_i) = Z_i\Sigma Z_i^{T} + \sigma^2 I$
- Therefore, $Y_i \sim MVN(X_i\beta, Z_i\Sigma Z_i^{T} + \sigma^2 I)$

We use the pdf of the multivariate normal distribution with the above parameters to derive the log-likelihood function. Below is the derivation:

\begin{align}
\frac{n_i}{2} \log(2\pi) - \frac{1}{2} \log \vert Z_i\Sigma Z_i^{T} + \sigma^2 I \vert - \frac{1}{2}(Y_i -X_i \beta)^{T}(Z_i\Sigma Z_i^{T} + \sigma^2 I)^{-1}(Y_i - X_i \beta)
\end{align}

We can use a Cholesky Decomposition to rewrite $\Sigma$ as $LL'$ where $L$ is a lower triangular matrix.

This gives us:

\begin{align}
\frac{n_i}{2} \log(2\pi) - \frac{1}{2} \log \vert Z_i LL^{T} Z_i^{T} + \sigma^2 I \vert - \frac{1}{2}(Y_i -X_i \beta)^{T}(Z_i\ LL^{T} Z_i^{T} + \sigma^2 I)^{-1}(Y_i - X_i \beta)
\end{align}

Now let $Z_iL = M$. This gives us:

\begin{align}
\frac{n_i}{2} \log(2\pi) - \frac{1}{2} \log \vert MM^{T} + \sigma^2 I \vert - \frac{1}{2}(Y_i -X_i \beta)^{T}(MM^{T} + \sigma^2 I)^{-1}(Y_i - X_i \beta)
\end{align}

We can take advantage of the easy + low rank properties to evaluate $(MM^{T} + \sigma^2 I)^{-1}$ using the Sherman-Woodbury formula, and $\vert MM^{T} + \sigma^2 I \vert$ using the matrix determinant lemma.

Starting with the Sherman-Woodbury formula, we evaluate $(MM^{T} + \sigma^2 I)^{-1}$ to be:

\begin{align}
(MM^{T} + \sigma^2 I)^{-1} 
&= \frac{1}{\sigma^2}I - \frac{1}{\sigma^4} M\big(I + \frac{1}{\sigma^2}M^{T}M\big)^{-1}M^{T}
\end{align}

It can be proved that $\big(I + \frac{1}{\sigma^2}M^{T}M\big)$ is positive semi-definite, and if we assume M is full rank, then it is positive-definite, which would allow us to apply another Cholesky Decomposition to this matrix. 
Let this Cholesky Decomposition be: $\big(I + \frac{1}{\sigma^2}M^{T}M\big) = AA^{T}$, where $A$ is a lower triangular matrix. This gives us:
\begin{align}
(MM^{T} + \sigma^2 I)^{-1} 
&= \frac{1}{\sigma^2}I - \frac{1}{\sigma^4} M\big(I + \frac{1}{\sigma^2}M^{T}M\big)^{-1}M^{T} \\
&= \frac{1}{\sigma^2}I - \frac{1}{\sigma^4} M(AA^{T})^{-1}M^{T}
\end{align}

Next we move onto the matrix determinant lemma and we evaluate $\vert MM^{T} + \sigma^2 I \vert$:
\begin{align}
\vert MM^{T} + \sigma^2 I \vert &= \sigma^{2n_i} \vert \big(I + \frac{1}{\sigma^2}M^{T}M\big) \vert \\
&=  \sigma^{2n_i} \vert AA^{T} \vert
\end{align}

Putting these together, we evaluate the log-likelihood to be:
\begin{align}
&\frac{n_i}{2} \log(2\pi) - \frac{n_i}{2} \log(\sigma^2) - \frac{1}{2}\log \vert AA^{T} \vert - \frac{1}{2 \sigma^2}(Y_i - X_i \beta)^{T}(Y_i - X_i \beta) + \frac{1}{2 \sigma^4}(Y_i - X_i \beta)^{T}M(AA^{T})^{-1}M^{T}(Y_i - X_i \beta) \\
&= \frac{n_i}{2} \log(2\pi) - \frac{n_i}{2} \log(\sigma^2) - \frac{1}{2}\log \vert AA^{T} \vert - \frac{1}{2 \sigma^2}(Y_i^{T}Y_i - 2Y_i^{T}X_i \beta + \beta^{T}X_i^{T}X_i \beta) + \frac{1}{2 \sigma^4}(Y_i - X_i \beta)^{T}M(AA^{T})^{-1}M^{T}(Y_i - X_i \beta) \\
&= \frac{n_i}{2} \log(2\pi) - \frac{n_i}{2} \log(\sigma^2) - \frac{1}{2}\log \vert AA^{T} \vert - \frac{1}{2 \sigma^2}(Y_i^{T}Y_i - 2Y_i^{T}X_i \beta + \beta^{T}X_i^{T}X_i \beta) + \frac{1}{2 \sigma^4}(A^{-1}L^{T}Z_i^T{}(Y_i - X \beta))^{T}(A^{-1}L^{T}Z_i^T{}(Y_i - X \beta)) \\
&= \frac{n_i}{2} \log(2\pi) - \frac{n_i}{2} \log(\sigma^2) - \frac{1}{2}\log \vert AA^{T} \vert - \frac{1}{2 \sigma^2}(Y_i^{T}Y_i - 2Y_i^{T}X_i \beta + \beta^{T}X_i^{T}X_i \beta) \\
&+ \frac{1}{2 \sigma^4}(Z_i^{T}Y_i - Z_i^{T}X \beta)^{T}(A^{-1}L^{T})^{T}(A^{-1}L^{T})(Z_i^{T}Y_i - Z_i^{T}X \beta) \\
&= \frac{n_i}{2} \log(2\pi) - \frac{n_i}{2} \log(\sigma^2) - \frac{1}{2}\log \vert AA^{T} \vert - \frac{1}{2 \sigma^2}(Y_i^{T}Y_i - 2Y_i^{T}X_i \beta + \beta^{T}X_i^{T}X_i \beta) \\ &+ ((Z_i^{T}Y_i - Z_i^{T}X \beta)^{T}L)(A^{T})^{-1}A^{-1}L^{T}(Z_i^{T}Y_i - Z_i^{T}X \beta)
\end{align}

Which leaves us with the final form we will utilize to write an efficient function designed to calculate this value. 

#### Q2: Start-up code

Use the following template to define a type LmmObs that holds an LMM datum  (𝐲𝑖,𝐗𝑖,𝐙𝑖) .

In [15]:
# define a type that holds LMM datum
struct LmmObs{T <: AbstractFloat}
# data
    y :: Vector{T}
    X :: Matrix{T}
    Z :: Matrix{T}
    # working arrays
    # whatever intermediate arrays you may want to pre-allocate
    #storage_n :: Vector{T}
    storage_qq :: Matrix{T}
    storage_qq2 :: Matrix{T}
    storage_qq3 :: Matrix{T}
    storage_nq :: Matrix{T}
    storage_nq2 :: Matrix{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    storage_q2  :: Vector{T}
    ztz :: Matrix{T}
    yty :: T
    xty :: Vector{T}
    xtx :: Matrix{T}
    ztx :: Matrix{T}
    zty :: Vector{T}
end

# constructor
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}) where T <: AbstractFloat
    #storage_n = similar(y)
    ztz = transpose(Z) * Z
    det = similar(ztz)
    storage_qq = similar(ztz)
    storage_qq2 = similar(ztz)
    storage_qq3 = similar(ztz)
    storage_nq = Matrix{T}(undef, n, q)
    storage_nq2 = Matrix{T}(undef, n, q)
    storage_p  = Vector{T}(undef, size(X, 2))
    storage_q  = Vector{T}(undef, size(Z, 2))
    storage_q2  = Vector{T}(undef, size(Z, 2))
    yty = transpose(y) * y
    xty = transpose(X) * y
    xtx = transpose(X) * X
    ztx = transpose(Z) * X
    zty = transpose(Z) * y
    
    LmmObs(y, X, Z, storage_qq, storage_qq2, storage_qq3, storage_nq, 
        storage_nq2, storage_p, storage_q, storage_q2,
        ztz, yty, xty, xtx, ztx, zty)
end

LmmObs

Write a function, with interface:

`logl!(obs, β, L, σ²)`

that evaluates the log-likelihood of the  $i$ -th datum. Here `L` is the lower triangular Cholesky factor from the Cholesky decomposition $\Sigma$=LL'. Make your code efficient in the $n_i >> q$  case. Think the intensive longitudinal measurement setting.

In [16]:
function logl!(
        obs :: LmmObs{T}, 
        β   :: Vector{T}, 
        L   :: Matrix{T}, 
        σ²  :: T) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)    
    # TODO: compute and return the log-likelihood

    #sleep(1e-3) # wait 1 ms as if your code takes 1ms
    
    mul!(obs.storage_qq, obs.ztz, L) 
    #ztzL
    BLAS.trmm!('L', 'L', 'T', 'N', AbstractFloat(1), L, obs.storage_qq) 
    #L'ztzL
    BLAS.axpy!(AbstractFloat(σ²), Matrix(I, q, q), obs.storage_qq) 
    # seems good but seems to be an issue with using float64 here
    # L'Z'ZL + σ²I 
    copy!(obs.storage_qq3, obs.storage_qq)
    LAPACK.potrf!('U', obs.storage_qq) 
    # extract A' = V from cholesky on M 
    
    ## term 3 ##
    
    mul!(obs.storage_qq2, transpose(obs.storage_qq), obs.storage_qq) 
    # V'V # cannot estimate this because it leads to a one point 
    # difference in true and computed values 

    
    # term 4
    mul!(obs.storage_p, obs.xtx, β) 
    (obs.yty - 2 * transpose(β) * obs.xty +  transpose(β) * obs.storage_p) 
    
    # term 5
    
    copy!(obs.storage_q, obs.zty)
    BLAS.gemv!('N', AbstractFloat(-1), obs.ztx, β, T(1), obs.storage_q)  
    # z'y - z'xβ 
    BLAS.trmv!('L', 'T', 'N', L, obs.storage_q) 
    # L'(z'y - z'xβ)
    
    
    copy!(obs.storage_q2, obs.storage_q)
    BLAS.trsv!('U', 'T', 'N', obs.storage_qq, obs.storage_q2) 
    # # V'^{-1} L'(z'y - z'xβ)
    
    BLAS.trsv!('U', 'N', 'N', obs.storage_qq, obs.storage_q2) 
    # V^{-1}V'^{-1} L'(zty - ztxβ)
    
    #scalar = dot(obs.storage_q, obs.storage_q2)
    # (y'z - β'x'z)L*V^{-1}V'^{-1} L'(zty - ztxβ) 
    

    return -0.5 * n * log(2π) - 0.5 * n * log(σ²) - 
    .5 * logdet(I + (1/σ²) * transpose(L) * obs.ztz * L) - 
    0.5 * (1/σ²) * (obs.yty - 2 * transpose(β) * obs.xty +  transpose(β) 
        * obs.storage_p) + 
    .5 * (1/σ²) * dot(obs.storage_q, obs.storage_q2) 
    
end

logl! (generic function with 1 method)

#### Q3: Correctness

Compare your result (both accuracy and timing) to the Distributions.jl package using following data.

In [17]:
using BenchmarkTools, Distributions, LinearAlgebra, Random

Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form an LmmObs object
obs = LmmObs(y, X, Z)

LmmObs{Float64}([-1.450910909560209, 1.5185224894450862, 5.265021705624027, 4.485272594164557, 0.6949699666429332, 1.7723256696372407, 1.1065838446466518, 3.7291668118296073, 4.288899999400642, 2.8241842645202406  …  4.058027151891635, 1.0909724390970443, 0.026692243086209766, -0.8927757653299448, 6.94725248926293, 3.519302085567343, 4.914007299083773, 2.1610206566690797, 1.857389542694909, 6.513818951020866], [1.0 0.6790633442371218 … 0.5400611947971554 -0.632040682052606; 1.0 1.2456776800889142 … -0.4818455756130373 0.6467830314674976; … ; 1.0 0.0733124748775436 … 0.6125080259511859 0.4181258283983667; 1.0 -1.336609049786048 … -0.18567490803712938 1.0745977099307227], [1.0 -1.0193326822839996 -0.15855601254314888; 1.0 1.7462667837699666 -0.4584376230657152; … ; 1.0 1.4843185594903878 0.42458303115266854; 1.0 0.3791714762820068 0.25150666970865837], [1361.4372929891945 142.38078230100288 132.84215761688665; 0.10458122679186291 1302.862124596768 68.50124271390091; 0.09757493665038086 0

This is the standard way to evaluate log-density of a multivariate normal, using the Distributions.jl package. Let's evaluate the log-likelihood of this datum.

In [18]:
μ  = X * β
Ω  = Z * Σ * transpose(Z) +  σ² * I
mvn = MvNormal(μ, Symmetric(Ω)) # MVN(μ, Σ)
logpdf(mvn, y)

-3256.179335805832

Check that your answer matches that from Distributions.jl

In [19]:
L = Matrix(cholesky(Σ).L)
logl!(obs, β, L, σ²)

-3256.1793358058267

You will lose all 15 + 30 + 30 = 75 points if the following statement throws `AssertionError.`

In [20]:
@assert logl!(obs, β, Matrix(cholesky(Σ).L), σ²) ≈ logpdf(mvn, y)

#### Q4: Efficiency

Benchmarking your code and compare to the Distributions.jl function logpdf.

In [21]:
# benchmark the `logpdf` function in Distribution.jl
bm1 = @benchmark logpdf($mvn, $y)

BenchmarkTools.Trial: 337 samples with 1 evaluation.
 Range (min … max):  937.169 μs … 822.393 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):       1.777 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):    14.481 ms ±  62.964 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▁                                                             
  ██▇▅▅▅▅▄▄▄▄▁▄▅▄▁▁▁▁▄▄▁▄▁▁▄▁▅▁▄▁▁▁▄▁▁▁▁▄▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▅
  937 μs        Histogram: log(frequency) by time        295 ms <

 Memory estimate: 31.52 KiB, allocs estimate: 3.

In [22]:
# benchmark your implementation
L = Matrix(cholesky(Σ).L)
bm2 = @benchmark logl!($obs, $β, $L, $σ²)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.598 μs … 180.595 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.300 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.516 μs ±   4.727 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄█    ▇▅                                                     
  ██▆▃▄███▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  1.6 μs          Histogram: frequency by time        19.5 μs <

 Memory estimate: 656 bytes, allocs estimate: 6.

In [26]:
# this is the points you'll get
clamp(median(bm1).time / median(bm2).time / 1000 * 30, 0, 30)

# when I previously checked this I got 24 points (I think due to the speed of my computer)
# moved to a place with worse wifi and maybe that's why the score went down (?) 

16.15336292974136

#### Q5: Memory

You want to avoid memory allocation in the "hot" function logl!. You will lose 1 point for each 1 KiB = 1024 bytes memory allocation. In other words, the points you get for this question is

In [24]:
clamp(30 - median(bm2).memory / 1024, 0, 30)

29.359375